In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import streetview # Thanks to https://github.com/cplusx/google-street-view-panorama-download
from skimage import io
import numpy as np
import os
import cv2
import pathlib

In [3]:
# Go to https://map.project-osrm.org/
# Draw your path and click the button: Export GPX file
# Then, put your file path here
file_path = "./route.gpx"

In [4]:
def get_lat_long(file_path: str) -> list:
    """Function to load the file and get latitudes and longitudes of the path"""
    with open(file_path) as f:
        content = f.readlines()[1]
        content = content.split("<trkseg>")[1]
        content = content.split("</trkseg>")[0]
        lat_longs = []
        for coords in content.split("lat")[1:]:
            lat, lon = coords.split(" ")[0][1:], coords.split(" ")[1][4:]
            lon = lon[:lon.find("/")]
            lat, lon = eval(lat), eval(lon)
            lat_longs.append((float(lat), float(lon)))
    return lat_longs

def check_path(path: str):
    """Make sure a path exists"""
    path = pathlib.Path(path)
    if not path.exists():
        path.mkdir(parents=True)

def download_images(coords: list, 
                    show_img: bool=True, 
                    img_dir: str='', 
                    video_dir: str='') -> None:
    """Function to download panorama images from coordinates list"""
    if img_dir:
        img_cnt = 0
        check_path(img_dir)
        
    if video_dir:
        check_path(video_dir)
    
    for lat, lon in coords:
        print(f"Downloading now lat={lat}, lon={lon}")
        panoids = streetview.panoids(lat=lat, lon=lon)
        panoid = panoids[0]['panoid']
        panorama = streetview.download_panorama_v3(panoid, zoom=3, disp=False)
        
        if show_img:
            plt.figure(figsize=(15, 15))
            plt.imshow(panorama)
            plt.axis('off')
            plt.show()
        
        # Save img
        if img_dir:
            # file name format: img_number_latitude_longitude.png
            plt.imsave(f"{img_dir}/img_{img_cnt}_{lat}_{lon}.jpg", panorama)
            img_cnt += 1
        
        # Save video
        if video_dir:
            if 'out' not in locals():
                h, w, _ = panorama.shape
                out = cv2.VideoWriter(f"{video_dir}/output.avi", cv2.VideoWriter_fourcc(*'XVID'), 5.0, (w, h))
            
            panorama = cv2.cvtColor(panorama, cv2.COLOR_RGB2BGR)
            panorama = cv2.flip(panorama, 1)
            out.write(panorama)
    
    if 'out' in locals():
        out.release()

In [ ]:
# Extract your coordinates from your file
coords = get_lat_long(file_path)

# Download panorama images from your coordinates
download_images(coords, show_img=False, img_dir='./images/', video_dir='./video/')